# **Data Collection**

## Objectives

* Fetch data from Kaggle for further processing

## Inputs

* Kaggle JSON file - the authentication token.

## Outputs

* Generate Dataset: inputs/datasets/brain_tumor_mri_dataset

## Additional Comments

* No comments 


---

# Change working directory

* The notebooks in a subfolder, therefore when running the notebook in the editor, we will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [1]:
import os
current_dir = os.getcwd()
current_dir

'/workspace/brain-tumor-detector/jupyter_notebooks'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [2]:
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [3]:
current_dir = os.getcwd()
current_dir

'/workspace/brain-tumor-detector'

# Install The Kaggle

In [4]:
# installing kaggle packages
!pip install kaggle

  Preparing metadata (setup.py) ... done
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105787 sha256=2b2ede12f74a867be731fd5318aa7717acc7e112ecafec243e184fb45ff73590
  Stored in directory: /workspace/.pyenv_mirror/pip_cache/wheels/46/d2/26/84d0a1acdb9c6baccf7d28cf06962ec80529fe1ad938489983
Successfully built kaggle


---

Change the Kaggle configuration directory to the current working directory and set permissions for the Kaggle authentication JSON.

In [5]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set the Kaggle Dataset and Download it.

In [6]:
KaggleDatasetPath = "masoudnickparvar/brain-tumor-mri-dataset"
DestinationFolder = "inputs/brain-tumor-mri-dataset"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}

Dataset URL: https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
License(s): CC0-1.0
100%|███████████████████████████████████████▊| 148M/149M [00:04<00:00, 40.3MB/s]
100%|████████████████████████████████████████| 149M/149M [00:04<00:00, 33.1MB/s]


In [7]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/brain-tumor-mri-dataset.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/brain-tumor-mri-dataset.zip')

# Data Preparation